# 16_Connecting_DTOs_to_Request_Body_with_ValidationPipe

## 1. Recap: Why We Needed DTOs

* Built-in pipes (like `ParseIntPipe`, `DefaultValuePipe`) are great for:

  * Route parameters (e.g. `/users/:id`)
  * Simple query strings (e.g. `?limit=10&page=1`)
* BUT: Request bodies usually contain **larger, structured objects** (like a full `User` object).
* We need a clean way to:

  * Validate many fields at once (name, email, gender, etc.)
  * Enforce types and formats (email format, min length, mandatory fields…)
* Solution: **DTOs + class-validator + ValidationPipe**

---

## 2. Goal of This Part

We want to:

1. Use our `CreateUserDto` class to validate the **request body**.
2. Connect that DTO to the `POST /users` route.
3. Use **`ValidationPipe`** so NestJS automatically validates the body and returns proper error responses.

---

## 3. The DTO We Already Created

We previously created a DTO like:

```ts
// users/dtos/create-user.dto.ts
import {
  IsString,
  IsEmail,
  IsOptional,
  IsNotEmpty,
  MinLength,
} from 'class-validator';

export class CreateUserDto {
  id: number; // will usually be auto-generated in real apps

  @IsString({ message: 'Name should be a string value' })
  @IsNotEmpty({ message: 'Name should not be empty' })
  @MinLength(3, { message: 'Name should have a minimum of three characters' })
  name: string;

  @IsString()
  @IsOptional()
  gender?: string;

  @IsString()
  @IsEmail()
  email: string;

  isMarried: boolean;
}
```

**Important rule:**

* The **shape** of the object sent in the request body must match the DTO:

  * Body should have: `id`, `name`, `email`, `isMarried`, and optionally `gender`.

---

## 4. Why We Need `class-transformer`

* We already use **`class-validator`** to add decorators like `@IsString()`, `@IsEmail()`, etc.
* But Nest also needs to **convert plain JSON into a class instance** of `CreateUserDto`.
* That’s where **`class-transformer`** comes in.

### 4.1. Install `class-transformer`

From your NestJS project root:

```bash
npm install class-transformer@0.5.1
```

* Version `0.5.1` works well with `class-validator@0.14.1`.
* Check `package.json` → dependencies should now contain:

  * `"class-validator": "0.14.1"`
  * `"class-transformer": "0.5.1"`

`class-transformer` is used internally by Nest’s `ValidationPipe` to:

* Transform request JSON into an **instance** of `CreateUserDto`.
* Run all the validators on that instance.

---

## 5. Connecting DTO to the Controller Method

We want to validate the body for this route:

```ts
POST /users
```

### 5.1. UsersController Before Validation

Initially, the controller might look like this:

```ts
import { Controller, Get, Param, ParseIntPipe, Post, Query, DefaultValuePipe, Body } from '@nestjs/common';
import { UsersService } from './users.service';

@Controller('users')
export class UsersController {
  @Get()
  getUsers(
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
  ) {
    const usersService = new UsersService();
    console.log(limit, page);
    return usersService.getAllUsers();
  }

  @Get(':id')
  getUserById(@Param('id', ParseIntPipe) id: number) {
    const userService = new UsersService();
    console.log(typeof id, id);
    return userService.getUserById(id);
  }

  @Post()
  createUser(@Body() user: any) {
    // Not validated yet
    return 'User created successfully';
  }
}
```

We now want:

* `@Body()` to be **validated** using `CreateUserDto`.

---

## 6. Enabling Validation with `ValidationPipe`

### 6.1. Import `ValidationPipe` and DTO

Update the imports in `users.controller.ts`:

```ts
import {
  Controller,
  Get,
  Param,
  ParseIntPipe,
  Post,
  Query,
  DefaultValuePipe,
  ValidationPipe,
  Body,
} from '@nestjs/common';
import { UsersService } from './users.service';
import { CreateUserDto } from './dtos/create-user.dto';
```

> Note: Make sure the class name and file path are correct (`CreateUserDto`, not `CreeateUserDto`).

### 6.2. Attach `ValidationPipe` to `@Body()` and Type the Parameter

```ts
@Post()
createUser(@Body(new ValidationPipe()) user: CreateUserDto) {
  // For now, just return a message
  return 'User created successfully';
}
```

What this does:

* `@Body(new ValidationPipe())`:

  * Reads the incoming JSON body.
  * Transforms it into an instance of `CreateUserDto` (using `class-transformer`).
  * Runs all the validators defined in `CreateUserDto` (using `class-validator`).
* If **any validation fails**:

  * NestJS automatically returns a **400 Bad Request** response.
  * The response body contains an array of validation errors.

---

## 7. Example: Invalid Request and Error Response

### 7.1. Send Invalid Body

Example body (via Postman/Insomnia/etc.):

```json
{
  "id": 1,
  "name": "",          
  "email": 26,          
  "isMarried": false
}
```

Problems:

* `name` is an **empty string**

  * Violates `@IsNotEmpty()`
  * Violates `@MinLength(3)`
* `email` is a **number**, not a string / email

  * Violates `@IsString()`
  * Violates `@IsEmail()`

### 7.2. What Response Do We Get?

Nest will return a **400 Bad Request** with a validation error array, for example:

* Error for `name`:

  * `"name should have a minimum of three characters"` (our custom message from `@MinLength`)
  * `"name should not be empty"` (from `@IsNotEmpty`)
* Error for `email`:

  * `"email must be an email"` (from `@IsEmail`)

So, **before** the controller tries to actually create a user, NestJS stops the request and reports errors.

---

## 8. Example: Valid Request

Body:

```json
{
  "id": 1,
  "name": "John Doe",
  "email": "john@example.com",
  "isMarried": false
}
```

* `name`:

  * String ✅
  * Not empty ✅
  * At least 3 chars ✅
* `email`:

  * String ✅
  * Valid email format ✅
* `gender` omitted → allowed because it’s `@IsOptional()` ✅

Response:

```json
"User created successfully"
```

If you add `"gender": "male"`, it still passes, because:

* `gender` is a string
* It’s optional, but can be present.

---

## 9. Full Example of UsersController (With Pipes + DTO Validation)

```ts
import {
  Controller,
  Get,
  Param,
  ParseIntPipe,
  Post,
  Query,
  DefaultValuePipe,
  ValidationPipe,
  Body,
} from '@nestjs/common';
import { UsersService } from './users.service';
import { CreateUserDto } from './dtos/create-user.dto';

@Controller('users')
export class UsersController {
  @Get()
  getUsers(
    @Query('limit', new DefaultValuePipe(10), ParseIntPipe) limit: number,
    @Query('page', new DefaultValuePipe(1), ParseIntPipe) page: number,
  ) {
    const usersService = new UsersService();
    console.log(limit, page);
    return usersService.getAllUsers();
  }

  @Get(':id')
  getUserById(@Param('id', ParseIntPipe) id: number) {
    const userService = new UsersService();
    console.log(typeof id, id);
    return userService.getUserById(id);
  }

  @Post()
  createUser(@Body(new ValidationPipe()) user: CreateUserDto) {
    // Later: call UsersService to actually persist the user
    // const userService = new UsersService();
    // userService.createUser(user);

    return 'User created successfully';
  }
}
```

---

## 10. Key Takeaways

* **DTOs** define the structure + validation rules for incoming data.
* **class-validator** gives us decorator-based validators (`@IsString`, `@IsEmail`, `@MinLength`, etc.).
* **class-transformer** helps `ValidationPipe` convert JSON into DTO instances.
* **ValidationPipe** ties everything together:

  * Attach it to `@Body()` (or globally) to automatically validate incoming data.
* If the data is invalid → NestJS returns `400 Bad Request` with detailed error messages.

This completes the core flow:

> Client → sends JSON body → `ValidationPipe` + DTO validate it → Controller method receives only **validated data**.

---